In [17]:
import numpy as np
import igl

V, F = igl.read_triangle_mesh("../meshes/bunny.obj")
NV = V.shape[0]
NF = F.shape[0]

def shrinkage(x, k):
    return np.maximum( 0, x - k ) - np.maximum( 0, -x - k )

rotData = dict(
    V=V,
    F=F,
    N=igl.per_vertex_normals(V, F)[:, :, None],
    L=igl.cotmatrix(V, F),
    VA=igl.massmatrix(V, F).diagonal(),
    cubeness = 4e-1,
    rho = 1e-4,
    ABSTOL = 1e-5,
    RELTOL = 1e-3,
    mu = 5,
    tao = 2, 
    maxIter_ADMM = 100
)

ARAPdata = dict(
    L = igl.cotmatrix(V, F),
    preF = [],
    K = igl.arap_rhs(V, F, 3, igl.ARAP_ENERGY_TYPE_SPOKES_AND_RIMS)
)

In [18]:
rotData['zAll'] = np.zeros((NV, 3, 1))
rotData['uAll'] = np.zeros((NV, 3, 1))
rotData['rhoAll'] = np.full((NV, 1), 1e-4)

# VF[NI[i]:NI[i+1]] is the face indices adjacent to V[i]
VF, NI = igl.vertex_triangle_adjacency(F, NV)
rotData['NI'] = NI

# hElist[NI[i]:NI[i+1]] are all the edges of triangles adjacent to V[i]
hElist = np.empty((VF.shape[0], 3, 2), dtype=np.int32)
hElist[:, :, 0] = F[VF, :]
hElist[:, :, 1] = F[VF][:, (1,2,0)]

rotData['hElist'] = hElist

# dVlist is the displacement of the edges 
dVlist = np.empty((VF.shape[0], 3, 3), dtype=np.float32)
dVlist[:, 0, :] = V[hElist[:, 0, 1]] - V[hElist[:, 0, 0]]
dVlist[:, 1, :] = V[hElist[:, 1, 1]] - V[hElist[:, 1, 0]]
dVlist[:, 2, :] = V[hElist[:, 2, 1]] - V[hElist[:, 2, 0]]

rotData['dVlist'] = dVlist

# WvalList is the diagonal value of the Laplacian (cotmartix)
Wvallist = np.empty((VF.shape[0], 3))
Wvallist[:, 0] = rotData['L'][hElist[:, 0, 0], hElist[:, 0, 1]]
Wvallist[:, 1] = rotData['L'][hElist[:, 1, 0], hElist[:, 1, 1]]
Wvallist[:, 2] = rotData['L'][hElist[:, 2, 0], hElist[:, 2, 1]]
rotData['Wvallist'] = Wvallist

In [19]:
def proj_SE3(S):
    U, X, V_t = np.linalg.svd(S) #the SVD of linalg gives you Vt
    
    #Calculate rotation matrix
    R = (U @ V_t).T
    if np.linalg.det(R) < 0:
        U[:, -1] *= -1
        R = (U @ V_t).T
    return R

In [20]:
from tqdm import tqdm
def fitRotation(U, rotData):
    RAll = np.zeros((NV, 3, 3))

    for ii in tqdm(range(NV)):
        z = rotData['zAll'][ii]
        u = rotData['uAll'][ii]
        n = rotData['N'][ii]
        rho = rotData['rhoAll'][ii]

        hE = rotData['hElist'][rotData['NI'][ii]:rotData['NI'][ii+1]]

        flatten_size = hE.shape[0] * hE.shape[1]
        hE = hE.reshape(flatten_size, 2, order='F')

        W = rotData['Wvallist'][rotData['NI'][ii]:rotData['NI'][ii+1]]
        W = np.diag(W.reshape(flatten_size, order='F'))
        dV = rotData['dVlist'][rotData['NI'][ii]:rotData['NI'][ii+1]]
        dV = dV.reshape(flatten_size, 3, order='F')
        dU = U[hE[:, 1]] - U[hE[:, 0]]

        Spre = dV.T @ W @ dU
        
        for k in range(rotData['maxIter_ADMM']):
            S = Spre + rho * (n @ (z-u).T)
            R = proj_SE3(S)
            z_old = z
            z = shrinkage(R @ n + u, rotData['cubeness'] * rotData['VA'][ii] / rho)
            u += R @ n - z
            
            r_norm = np.linalg.norm(z - R @ n)
            s_norm = np.linalg.norm(-rho * (z - z_old))
            
            if r_norm > rotData['mu'] * s_norm:
                rho *= rotData['tao']
                u /= rotData['tao']
            elif s_norm > rotData['mu'] * r_norm:
                rho /= rotData['tao']
                u *= rotData['tao']
            
            eps_pri = np.sqrt(6) * rotData['ABSTOL'] + rotData['RELTOL'] * np.maximum(np.linalg.norm(R @ n), np.linalg.norm(z))
            eps_dual = np.sqrt(3) * rotData['ABSTOL'] + rotData['RELTOL'] * np.linalg.norm(rho * u)
            
            if r_norm < eps_pri and s_norm < eps_dual:
                rotData['zAll'][ii] = z
                rotData['uAll'][ii] = u
                rotData['rhoAll'][ii] = rho
                RAll[ii] = R
                break
    return RAll

In [21]:
maxIter = 50
b = np.array([999])
U = V.copy()
bc = V[b, :]
from scipy.sparse import csc_matrix
Aeq = csc_matrix((0, 0))
Beq = np.array([])
for iter in range(maxIter):
    RAll = fitRotation(U, rotData)
    Rcol = RAll.reshape(NV * 3 * 3, 1, order='F')
    Bcol = ARAPdata['K'] @ Rcol
    B = Bcol.reshape(int(Bcol.shape[0] / 3), 3, order='F')
    finish, U = igl.min_quad_with_fixed(ARAPdata['L'], B, b, bc, Aeq, Beq, False)
    igl.write_triangle_mesh("result.obj", U, F)

100%|██████████| 6172/6172 [00:01<00:00, 5688.48it/s]
